In [8]:
%cd ..
%load_ext autoreload
%autoreload 2

/Users/rlm/Desktop/Code/open_deep_research/src


/Users/rlm/Desktop/Code/open_deep_research/open-deep-research-env/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [6]:
! pip install -U -q open-deep-research


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [7]:
import open_deep_research   
print(open_deep_research.__version__) 

0.0.11


In [9]:
from IPython.display import Image, display
from langgraph.types import Command
from langgraph.checkpoint.memory import MemorySaver
from open_deep_research.graph import builder

In [ ]:
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [ ]:
display(Image(graph.get_graph(xray=1).draw_mermaid_png()))

In [69]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# Set the API keys used for any model or search tool selections below, such as:
_set_env("OPENAI_API_KEY")
_set_env("ANTHROPIC_API_KEY")
_set_env("TAVILY_API_KEY")
_set_env("GROQ_API_KEY")
_set_env("PERPLEXITY_API_KEY")

In [11]:
import uuid 
from IPython.display import Markdown

REPORT_STRUCTURE = """Use this structure to create a report on the user-provided topic:

1. Introduction (no research needed)
   - Brief overview of the topic area

2. Main Body Sections:
   - Each section should focus on a sub-topic of the user-provided topic
   
3. Conclusion
   - Aim for 1 structural element (either a list of table) that distills the main body sections 
   - Provide a concise summary of the report"""

# Claude 3.7 Sonnet for planning with perplexity search
thread = {"configurable": {"thread_id": str(uuid.uuid4()),
                           "search_api": "perplexity",
                           "planner_provider": "anthropic",
                           "planner_model": "claude-3-7-sonnet-latest",
                           # "planner_model_kwargs": {"temperature":0.8}, # if set custom parameters
                           "writer_provider": "anthropic",
                           "writer_model": "claude-3-5-sonnet-latest",
                           # "writer_model_kwargs": {"temperature":0.8}, # if set custom parameters
                           "max_search_depth": 2,
                           "report_structure": REPORT_STRUCTURE,
                           }}

# DeepSeek-R1-Distill-Llama-70B for planning and llama-3.3-70b-versatile for writing
thread = {"configurable": {"thread_id": str(uuid.uuid4()),
                           "search_api": "tavily",
                           "planner_provider": "groq",
                           "planner_model": "deepseek-r1-distill-llama-70b",
                           "writer_provider": "groq",
                           "writer_model": "llama-3.3-70b-versatile",
                           "report_structure": REPORT_STRUCTURE,
                           "max_search_depth": 1,}
                           }

# Fast config (less search depth) with o3-mini for planning and Claude 3.5 Sonnet for writing
thread = {"configurable": {"thread_id": str(uuid.uuid4()),
                           "search_api": "tavily",
                           "planner_provider": "openai",
                           "planner_model": "o3-mini",
                           "writer_provider": "anthropic",
                           "writer_model": "claude-3-5-sonnet-latest",
                           "max_search_depth": 1,
                           "report_structure": REPORT_STRUCTURE,
                           }}

# Create a topic
topic = "Overview of the AI inference market with focus on Fireworks, Together.ai, Groq"

# Run the graph until the interruption
async for event in graph.astream({"topic":topic,}, thread, stream_mode="updates"):
    if '__interrupt__' in event:
        interrupt_value = event['__interrupt__'][0].value
        display(Markdown(interrupt_value))

Please provide feedback on the following report plan. 
                        

Section: Introduction
Description: A brief overview of the AI inference market, its growth, and the importance of efficient inference solutions. Introduce Fireworks, Together.ai, and Groq as key players in the market.
Research needed: No


Section: Market Overview and Trends
Description: An analysis of the overall AI inference market including key trends, challenges, and drivers such as performance demands, cost efficiency, scalability, and adoption by businesses.
Research needed: Yes


Section: Detailed Analysis of Providers
Description: A deep dive into the specifics of Fireworks, Together.ai, and Groq. This section will cover performance metrics, scalability, pricing models, and developer experience for each provider.
Research needed: Yes


Section: Comparative Evaluation
Description: A side-by-side comparison of the three providers highlighting key differences and similarities in terms of performance, cost-effectiveness, scalability, and overall value.
Research needed: Yes


Section: Conclusion and Recommendations
Description: A concise summary of the main findings, including a distilled table or list of key insights from the analysis and recommendations based on the comparative evaluation.
Research needed: No


                        
Does the report plan meet your needs?
Pass 'true' to approve the report plan.
Or, provide feedback to regenerate the report plan:

In [12]:
# Pass feedback to update the report plan  
async for event in graph.astream(Command(resume="Include individual sections for Together.ai, Groq, and Fireworks with revenue estimates (ARR)"), thread, stream_mode="updates"):
    if '__interrupt__' in event:
        interrupt_value = event['__interrupt__'][0].value
        display(Markdown(interrupt_value))

Please provide feedback on the following report plan. 
                        

Section: Introduction
Description: Provides a brief overview of the AI inference market and introduces the key players: Fireworks AI, Together.ai, and Groq.
Research needed: No


Section: Market Overview
Description: An analysis of the overall AI inference market, including trends, demand, cost-efficiency, and scalability factors within the sector.
Research needed: Yes


Section: Fireworks AI Analysis with Revenue Estimates
Description: A focused look at Fireworks AI covering its performance, cost models, scalability, developer experience, and estimated annual recurring revenue (ARR).
Research needed: Yes


Section: Together.ai Analysis with Revenue Estimates
Description: A detailed examination of Together.ai concentrating on its training capabilities, cost efficiency in GPU usage, enterprise features, and revenue (ARR) estimates.
Research needed: Yes


Section: Groq Analysis with Revenue Estimates
Description: A section dedicated to Groq, reviewing its unique hardware optimizations for inference, pricing models, performance metrics, and its estimated ARR.
Research needed: Yes


Section: Comparative Analysis
Description: A side-by-side comparison of Fireworks AI, Together.ai, and Groq, highlighting their strengths, weaknesses, cost-effectiveness, and revenue implications to help stakeholders decide.
Research needed: Yes


Section: Conclusion
Description: Summarizes the report findings, distills the main points into a concise table or list, and provides a final assessment of the market and the key players.
Research needed: No


                        
Does the report plan meet your needs?
Pass 'true' to approve the report plan.
Or, provide feedback to regenerate the report plan:

In [13]:
# Pass True to approve the report plan 
async for event in graph.astream(Command(resume=True), thread, stream_mode="updates"):
    print(event)
    print("\n")

{'human_feedback': None}


{'build_section_with_web_research': {'completed_sections': [Section(name='Together.ai Analysis with Revenue Estimates', description='A detailed examination of Together.ai concentrating on its training capabilities, cost efficiency in GPU usage, enterprise features, and revenue (ARR) estimates.', research=True, content="Based on the provided source material, I'll write a comprehensive analysis of Together.ai with revenue estimates:\n\n## Together.ai Analysis with Revenue Estimates\n\nTogether.ai has demonstrated remarkable growth since its founding in 2022, reaching significant revenue milestones in rapid succession. The company achieved $44 million in annual recurring revenue (ARR) by April 2024, representing a staggering 2,095% year-over-year increase, and further accelerated to $100 million ARR by July 2024 [1]. Current estimates suggest Together.ai's ARR has reached approximately $130 million as of early 2025 [2].\n\nThe company's growth is driven by its p

In [14]:
final_state = graph.get_state(thread)
report = final_state.values.get('final_report')
Markdown(report)

# AI Inference Market: Analyzing Fireworks, Together.ai, and Groq

The AI inference market is undergoing rapid transformation as new providers emerge with innovative approaches to deploying and scaling AI models. This report examines three key players - Fireworks AI, Together.ai, and Groq - who represent distinct strategies in addressing the growing demand for AI inference solutions. While Fireworks AI focuses on optimizing conventional GPU infrastructure for cost-efficiency, Together.ai emphasizes comprehensive training capabilities, and Groq differentiates through custom hardware designed for maximum speed. Understanding these providers' unique approaches and market positions is crucial for organizations navigating the evolving landscape of AI deployment options.

Here's my synthesis of the market overview section focused on the AI inference market with a focus on Fireworks, Together.ai, and Groq:

## Market Overview

The AI inference market is experiencing rapid growth, projected to reach between $780 billion and $990 billion by 2027, with a compound annual growth rate of 19.2% [1]. This expansion is driven by increasing demand for real-time processing capabilities and the need for efficient AI model deployment across industries.

In the competitive landscape, each provider has distinct advantages. Groq leads in raw inference speed, achieving up to 250 tokens per second with their custom hardware [2]. Fireworks AI differentiates itself through cost-effective inference and consistent performance across varying workload sizes, particularly excelling at processing large context windows up to 10K tokens [3]. Together.ai focuses on AI training capabilities with access to over 10,000 GPUs, though their inference speeds can be variable under heavy loads [4].

The market is seeing significant innovation in infrastructure optimization, with providers taking different approaches. While some like Groq utilize custom chips for maximum speed, others like Fireworks AI focus on optimizing conventional GPU infrastructure for better cost-efficiency and flexibility [3]. This diversification in approaches helps serve different customer needs, from enterprises requiring maximum speed to startups seeking cost-effective solutions.

### Sources
[1] https://www.globenewswire.com/news-release/2025/04/21/3064502/0/en/AI-Inference-Market-Forecast-Report-to-2030-with-Case-Studies-of-Intel-Siemens-Healthineers-Nvidia-Eleuther-AI.html
[2] https://koonka.ai/fireworks-ai-vs-together-ai/
[3] https://medium.com/friendliai/a-comparative-analysis-of-ai-api-providers-based-on-llama-3-1-70b-a9d89c52bfd1
[4] https://www.peerspot.com/products/comparisons/fireworks-ai_vs_together-inference

## Fireworks AI Analysis with Revenue Estimates

Fireworks AI has established itself as a leading platform for serving open-source AI models with competitive performance and pricing. The company offers multiple deployment options including serverless inference, on-demand deployments, and enterprise reserved GPUs [1].

Key strengths include superior performance, with their proprietary FireAttention serving stack delivering up to 2.5x faster generation speeds and 3.5x higher throughput compared to competitors like HuggingFace TGI on the same hardware [2]. Their on-demand deployments achieve 6x lower costs while maintaining high performance.

The platform supports a comprehensive model ecosystem including popular open-source models like Llama, DeepSeek, and Mixtral across text, vision, audio and embedding modalities [3]. Their serverless models feature industry-leading speeds of up to 300 tokens/second for models like Mixtral, with rate limits of 600 requests/minute [2].

Pricing is competitive and usage-based, with options like:
- Serverless inference with per-token pricing starting at $0.20 per million tokens
- On-demand GPU deployments at $3.89 per hour for A100s
- Enterprise plans with custom pricing based on reserved capacity [2]

While private financial data is limited, given their technology advantages and major customers like Tome, Quora and Sourcegraph [2], estimated annual recurring revenue (ARR) is likely in the $10-20M range based on market comparables.

### Sources
[1] https://docs.fireworks.ai/getting-started/introduction 
[2] https://fireworks.ai/blog/spring-update-faster-models-dedicated-deployments-postpaid-pricing
[3] https://artificialanalysis.ai/providers/fireworks

Based on the provided source material, I'll write a comprehensive analysis of Together.ai with revenue estimates:

## Together.ai Analysis with Revenue Estimates

Together.ai has demonstrated remarkable growth since its founding in 2022, reaching significant revenue milestones in rapid succession. The company achieved $44 million in annual recurring revenue (ARR) by April 2024, representing a staggering 2,095% year-over-year increase, and further accelerated to $100 million ARR by July 2024 [1]. Current estimates suggest Together.ai's ARR has reached approximately $130 million as of early 2025 [2].

The company's growth is driven by its position as a developer-focused platform for AI infrastructure, offering both GPU compute resources and comprehensive tooling for training and deploying open-source AI models. Together.ai differentiates itself through token-based pricing rather than hourly rates, which appeals particularly to startups with variable workloads [3].

Together.ai's recent $228.5 million in funding and $1.25 billion valuation reflect investor confidence in its business model, which maintains approximately 45% gross margins despite operating in the competitive cloud GPU market [4]. The company has secured strategic partnerships with major industry players like Dell Technologies and Hypertec Cloud to expand its infrastructure capabilities [5].

### Sources
[1] https://www.arr.club/signal/together-ai-arr-at-100m-in-less-than-10-months
[2] https://sacra.com/c/together-ai/
[3] https://www.together.ai/enterprise
[4] https://sacra.com/c/together-ai/
[5] https://www.dell.com/en-us/dt/corporate/newsroom/announcements/detailpage.press-releases~usa~2025~03~together-ai-taps-dell-technologies-to-scale-ai-acceleration-cloud-for-the-enterprise.htm

Here's my synthesized section on Groq's Analysis with Revenue Estimates:

## Groq Analysis with Revenue Estimates

Groq has demonstrated impressive technological capabilities with its Language Processing Unit (LPU) for AI inference, though its revenue figures remain modest as it scales up operations. The company's estimated annual revenue is currently $46.6M, with revenue per employee of approximately $116,842 [1][2]. Groq has raised significant funding totaling $1B and achieved a valuation of $2.8B as of August 2024 [1][2].

The company's competitive positioning centers on its unique TSP (Tensor Streaming Processor) architecture that delivers superior low-latency performance for AI inference workloads. Groq's LPU has significant cost advantages in manufacturing, with wafer costs estimated at under $6,000 on 14nm compared to Nvidia's $16,000 for 5nm H100 wafers. This translates to a 70% lower raw silicon cost per token, though system-level costs narrow the total cost of ownership advantage to around 40% [3].

Groq monetizes its technology through multiple revenue streams: cloud-based inference API services, hardware leasing and colocation services, and direct hardware sales for data centers [4]. The company has secured major enterprise deployments in Norway and Saudi Arabia, indicating growing commercial traction [4]. While still an emerging player, Groq's aggressive pricing and performance advantages position it to potentially capture a meaningful share of the rapidly growing AI inference market.

### Sources
[1] https://growjo.com/company/Groq
[2] https://compworth.com/company/groq 
[3] https://sacra.com/c/groq/
[4] https://www.chipstrat.com/p/groqs-business-model-part-3-competing

## Comparative Analysis

Based on the provided source material, here is a side-by-side comparison of Fireworks AI, Together.ai, and Groq highlighting their key strengths and differentiators:

Performance:
- Groq leads in raw inference speed, hitting 250-450 tokens/second with their custom LPU hardware
- Fireworks AI achieves around 70-100 tokens/second with optimized software on NVIDIA GPUs
- Together.ai delivers 80-90 tokens/second using GPU clusters, with strong reliability

Cost-Effectiveness:
- Together.ai offers competitive pricing at $0.10-0.20 per million tokens for base models
- Fireworks AI focuses on cost-efficient inference with rates around $0.20-0.50 per million tokens
- Groq balances premium performance with midrange pricing of $0.60-0.70 per million tokens

Key Advantages:
- Fireworks AI: Strong developer experience, fast API deployment, extensive model support, and consistent performance at scale
- Together.ai: Best for AI training, robust fine-tuning capabilities, private model ownership, and enterprise features
- Groq: Fastest raw inference speeds, custom hardware optimization, and ultra-low latency for real-time applications

Best Suited For:
- Fireworks AI: Startups and teams needing reliable, cost-effective inference with good developer tools
- Together.ai: Enterprises requiring training capabilities, model customization, and scalable infrastructure
- Groq: Applications demanding maximum speed and lowest possible latency

### Sources:
[1] https://koonka.ai/fireworks-ai-vs-together-ai/
[2] https://artificialanalysis.ai/models/llama-3-instruct-70b/providers
[3] https://www.helicone.ai/blog/llm-api-providers
[4] https://sacra.com/c/groq/

## Conclusion

The AI inference market is experiencing rapid evolution with three distinct approaches emerging. Groq leads in raw performance through custom hardware, achieving speeds up to 450 tokens/second, while maintaining healthy margins despite significant infrastructure investments. Together.ai has demonstrated remarkable revenue growth to $130M ARR through its developer-friendly platform and comprehensive training capabilities. Fireworks AI captures the middle ground with optimized performance and cost-efficiency.

| Metric | Groq | Together.ai | Fireworks AI |
|--------|------|-------------|--------------|
| Revenue | $46.6M | $130M | $10-20M |
| Speed | 250-450 tok/s | 80-90 tok/s | 70-100 tok/s |
| Cost | $0.60-0.70/M tok | $0.10-0.20/M tok | $0.20-0.50/M tok |
| Key Strength | Custom Hardware | Training Focus | Cost-Efficiency |
| Target Market | Enterprise | Developers | Startups |

As the market matures, expect further specialization among providers, with Groq likely expanding its hardware advantage, Together.ai strengthening its training capabilities, and Fireworks AI optimizing its middle-market position. Success will increasingly depend on matching specific customer needs rather than competing solely on speed or cost.

Trace: 

https://smith.langchain.com/public/d32aad4d-e5b8-49f9-a082-4987de5a8970/r